In [12]:
# import dependencies
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date, datetime, timedelta, time
import re
import numpy as np

In [13]:
# open and read the xml file
with open('export.xml', 'r') as f:
    data = f.read()

bs_data = BeautifulSoup(data, 'xml')

# find all of the hr data
hr_data = bs_data.find_all(type='HKQuantityTypeIdentifierHeartRate')

In [14]:
# create the lists we want to hold the data
date_creation = []
time_creation = []
hr_value = []
hr_bin = []
time_bin = []

# create a dictionary with the data
hr_dict = {
    'date_created': date_creation,
    'time_created': time_creation,
    'hr_values': hr_value,
    'hr_bin': hr_bin,
    'time_bin': time_bin
}

# loop through the data to pull the data we need
for idx, hr in enumerate(hr_data):
    creation_list = hr['creationDate']
    creation = creation_list.split()
    date_creation.append(creation[0])
    time_creation.append(creation[1])

    hr_value.append(float(hr['value']))

# create bins for hr values
# high > 100
# low < 60
# normal 60-100
for i in hr_value:
    if i > 100:
        hr_bin.append('high')
    elif i < 60:
        hr_bin.append('low')
    else:
        hr_bin.append('normal')

# bins for each time range
def time_in_range(start_time, end_time, creation_time):
    return start_time.strftime('%H:%M:%S') <= creation_time.strftime('%H:%M:%S') <= end_time.strftime('%H:%M:%S')

# the start and end times
# before first class 0
start_time_0 = time(6, 45, 0)
end_time_0 = time(9, 25, 0)
# 5th Grade
start_time_1 = time(9, 30, 0)
end_time_1 = time(10, 15, 0)
# 4th Grade
start_time_2 = time(10, 20, 0)
end_time_2 = time(11, 5, 0)
# lunch
start_time_3 = time(11, 5, 0)
end_time_3 = time(11, 35, 0)
# kinder
start_time_4 = time(11, 35, 0)
end_time_4 = time(12, 20, 0)
# 1st grade
start_time_5 = time(12, 25, 0)
end_time_5 = time(13, 10, 0)
# 3rd grade
start_time_6 = time(13, 15, 0)
end_time_6 = time(14, 0, 0)
# 2nd grade
start_time_7 = time(14, 5, 0)
end_time_7 = time(14, 50, 0)
# end of day
start_time_8 = time(14, 50, 0)
end_time_8 = time(16, 0, 0)
# out of school
start_time_9 = time(16, 0, 0)
end_time_9 = time(23, 59, 59)


for i in hr_dict['time_created']:
    creation_time = i
    time_dt = datetime.strptime(creation_time, '%H:%M:%S')
    time_dt.strftime('%H:%M:%S')
    if time_in_range(start_time_0, end_time_0, time_dt) == True:
        time_bin.append(0)
    elif time_in_range(start_time_1, end_time_1, time_dt) == True:
        time_bin.append(1)
    elif time_in_range(start_time_2, end_time_2, time_dt) == True:
        time_bin.append(2)
    elif time_in_range(start_time_3, end_time_3, time_dt) == True:
        time_bin.append(3)
    elif time_in_range(start_time_4, end_time_4, time_dt) == True:
        time_bin.append(4)
    elif time_in_range(start_time_5, end_time_5, time_dt) == True:
        time_bin.append(5)
    elif time_in_range(start_time_6, end_time_6, time_dt) == True:
        time_bin.append(6)
    elif time_in_range(start_time_7, end_time_7, time_dt) == True:
        time_bin.append(7)
    elif time_in_range(start_time_8, end_time_8, time_dt) == True:
        time_bin.append(8)
    elif time_in_range(start_time_9, end_time_9, time_dt) == True:
        time_bin.append(9)
    else:
        time_bin.append(10)

In [15]:
# create a df
hr_df = pd.DataFrame(hr_dict)

In [16]:
hr_df

,date_created,time_created,hr_values,hr_bin,time_bin
0,2015-12-25,14:06:03,65.0,normal,7
1,2015-12-25,14:25:52,107.0,high,7
2,2015-12-25,14:33:08,67.0,normal,7
3,2015-12-25,14:43:08,97.0,normal,7
4,2015-12-25,14:52:27,61.0,normal,8
...,...,...,...,...,...
118832,2022-05-04,20:40:42,80.0,normal,9
118833,2022-05-04,20:47:21,82.0,normal,9
118834,2022-05-04,20:51:44,90.0,normal,9
118835,2022-05-04,21:00:31,106.0,high,9


Getting the data for the school days

In [17]:
# Returns whether day is in the range [start, end]
def date_in_range(start, end, day):
    return start <= day <= end

# the start and end days
start = datetime(2022, 3, 28)
end = datetime(2022, 5, 4)

In [55]:
date_creation = []
time_creation = []
hr_value = []
hr_bin = []
time_bin = []
day_week = []

# grabbing all the rows for the selected time range
for idx, row in hr_df.iterrows():
    day = row['date_created']
    day_dt = datetime.strptime(day, '%Y-%m-%d')
    if date_in_range(start, end, day_dt) == True:
        day_week.append(datetime.weekday(day_dt))
        date_creation.append(day)
        time_creation.append(row['time_created'])
        hr_value.append(row['hr_values'])
        hr_bin.append(row['hr_bin'])
        time_bin.append(row['time_bin'])

school_dict = {
    'date_created': date_creation,
    'time_created': time_creation,
    'hr_values': hr_value,
    'hr_bin': hr_bin,
    'time_bin': time_bin,
    'day_week': day_week
}

school_df = pd.DataFrame(school_dict)

# removing all the weekend days
school_df = school_df[school_df['day_week'] < 5]
school_df

,date_created,time_created,hr_values,hr_bin,time_bin,day_week
0,2022-03-28,18:44:22,96.0,normal,9,0
1,2022-03-28,18:45:04,105.0,high,9,0
2,2022-03-28,18:45:09,98.0,normal,9,0
3,2022-03-28,18:45:14,100.0,normal,9,0
4,2022-03-28,18:45:19,100.0,normal,9,0
...,...,...,...,...,...,...
7182,2022-05-04,20:40:42,80.0,normal,9,2
7183,2022-05-04,20:47:21,82.0,normal,9,2
7184,2022-05-04,20:51:44,90.0,normal,9,2
7185,2022-05-04,21:00:31,106.0,high,9,2


Get all the info for each day within our range of dates

In [30]:
unique_dates = []
daily_info = []

# find each day within the school_df
for i in school_df['date_created']:
    if i not in unique_dates:
        unique_dates.append(i)

# find the avg_hr, min_hr, max_hr, and median_hr for each day
for i in unique_dates:
    hrs = []
    for idx, row in school_df.iterrows():
        if i == row['date_created']:
            hrs.append(row['hr_values'])
    avg_hr = format(np.mean(hrs), '.2f')
    min_hr = np.min(hrs)
    max_hr = np.max(hrs)
    median_hr = np.median(hrs)
    
    daily_dicts = {
        'day': i,
        'avg_hr': avg_hr,
        'min_hr': min_hr,
        'max_hr': max_hr,
        'median_hr': median_hr
    }

    daily_info.append(daily_dicts)